In [ ]:
!pip install datasets

Generate incomplete classfier dataset from bookcorpus dataset

In [9]:
from datasets import load_dataset
import pandas as pd
import random
random.seed(1)  # For consistency between the runs

In [10]:
class GenerateDataset():
  def __init__(self, dataset,length,is_train=True,min_length=8, max_length=20):
    self.dataset = load_dataset(dataset) # Can take a lot of time depending upon dataset
    if is_train:
      self.dataset=self.dataset["train"]
    # mixed dataset
    self.dataset = self.dataset.shuffle(seed=55)
    self.length=length
    self.min_length=min_length
    self.max_length=max_length
    # self.dataset=self.get_dataset()
    # self.__dataset=self.preprocess(self.dataset)  # For debugging purposes only
    self.__dataset=self.preprocess_test(self.dataset) # directly preprocess the dataset 
    self.dataset=self.gen_incomplete(self.__dataset)

  @property
  def complete_dataset(self):
    return self.__dataset

  def get_dataset(self,param=None):
    if param==None:
      premlim_li=random.choices(self.dataset,k=self.length)
    else:
      premlim_li=random.choices(self.dataset,k=param)
    return premlim_li
  
  def preprocess_test(self,li):
    buffer = []
    n = self.length
    for i in li:
      if len(i["text"].strip().split()) > self.min_length and i["text"] not in buffer and "**" not in i["text"]:
        buffer.append(i["text"])
        n -= 1
      if n == 0:
        break
    return buffer
  
  # def redundant_character(self,sentence,check=[" ","'","''"]):
  #   arr=sentence["text"].strip().split()
  #   count=len(arr)
  #   for i in arr:
  #     if i in check:
  #       count-=1
  #   return count

  # def preprocess(self,li):
  #   buffer=[]
  #   count=0
  #   max_iter=1000
  #   while True:
  #     max_iter-=1
  #     if max_iter==0:
  #       if len(buffer)<self.length:
  #         temp=self.get_dataset(self.length-len(buffer))
  #         buffer.extend([i["text"] for i in temp])
  #       break
  #     for i in li:
  #       if self.redundant_character(i)<=self.min_length:
  #         count+=1
  #       else:
  #         buffer.append(i["text"])
  #     if count==0 or len(buffer)==self.length:
  #       break
  #     else:
  #       li=self.get_dataset(count)
  #       count=0
  #   return buffer

  def gen_incomplete(self,pre_li):
    buffer=[]
    for i in pre_li:
      arr=i.strip().split()
      if len(arr) > self.max_length:
        buffer.append(" ".join(arr[:8]))
      else:
        buffer.append(" ".join(arr[:(len(arr)//2)]))

    return buffer

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    return self.dataset[idx]

  def toCSV(self):
    df=pd.DataFrame([i for i in self.dataset],columns=["x_i"])
    df.to_csv("incomplete_dataset.csv")


In [4]:
data=GenerateDataset("bookcorpus",10)
data.toCSV()

NameError: name 'GenerateDataset' is not defined

In [24]:
data[95]

'um , yeah , i got a flat'

In [13]:
# Load the dataset
df = pd.read_csv('incomplete_dataset.csv')

# Replace all instances of "`" with an empty string in the entire DataFrame or a specific column
df_cleaned = df.applymap(lambda x: x.replace("`", "") if isinstance(x, str) else x)

# replace all instances of """ with an empty string in the entire DataFrame or a specific column
df_cleaned = df_cleaned.applymap(lambda x: x.replace('""', "") if isinstance(x, str) else x)

# If you want to do this for a specific column, let's say 'text_column'
# df['text_column'] = df['text_column'].str.replace("``", "")

# Save the cleaned dataset
df_cleaned.to_csv('cleaned_dataset.csv', index=False)

# Display the cleaned data
df_cleaned.head()

C:\Users\YEH\AppData\Local\Temp\ipykernel_12484\884775774.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.applymap(lambda x: x.replace("`", "") if isinstance(x, str) else x)
C:\Users\YEH\AppData\Local\Temp\ipykernel_12484\884775774.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df_cleaned.applymap(lambda x: x.replace('""', "") if isinstance(x, str) else x)


,Unnamed: 0,x_i
0,0,"terry said , let me"
1,1,i have forever dreamed of belonging to callias
2,2,kyle started yipping then changed to
3,3,and i happen to know this side of
4,4,i think he feels ...


In [ ]:
!pip install transformers
!pip install torch
!pip install SentencePiece
!pip install tqdm

Check GPU availability

In [5]:
import torch

# Check if GPU is available
print(torch.cuda.is_available())  # This should return True if a GPU is available
print(torch.version.cuda)

# If True, print details about the GPU
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"Device Count: {torch.cuda.device_count()}")

# Check memory usage on GPU
allocated_memory = torch.cuda.memory_allocated() / 1024 ** 2  # Convert to MB
reserved_memory = torch.cuda.memory_reserved() / 1024 ** 2    # Convert to MB

print(f"Allocated memory: {allocated_memory:.2f} MB")
print(f"Reserved memory: {reserved_memory:.2f} MB")

True
12.1
GPU Device: NVIDIA GeForce RTX 3060 Laptop GPU
Device Count: 1
Allocated memory: 0.00 MB
Reserved memory: 5458.00 MB


Sentences generation and classfier dataset completion

In [8]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPTNeoForCausalLM
from transformers import XLNetTokenizer, XLNetLMHeadModel
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from transformers import CTRLTokenizer, CTRLLMHeadModel
from transformers import ReformerTokenizer, ReformerModelWithLMHead
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load dataset
test_dataset_path = 'test_dataset.csv'
cleaned_dataset_path = 'cleaned_dataset.csv'
cleaned_df = pd.read_csv(cleaned_dataset_path)
test_df = pd.read_csv(test_dataset_path)

# Set the device to GPU or CPU    
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_model_token(model_name, device):
    match model_name:
        case "GPT2":
            model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
            tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
            saved_path = "completed_dataset_GPT2.csv"
        case "XLNet":
            model = XLNetLMHeadModel.from_pretrained("xlnet-base-cased").to(device)
            tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
            saved_path = "completed_dataset_XLNet.csv"
        case "BART":
            model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)
            tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
            saved_path = "completed_dataset_BART.csv"
        case "T5":
            model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
            tokenizer = T5Tokenizer.from_pretrained("t5-small")
            saved_path = "completed_dataset_T5.csv"
        case "Pegasus":
            model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum").to(device)
            tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
            saved_path = "completed_dataset_Pegasus.csv"
        case "CTRL":
            model = CTRLLMHeadModel.from_pretrained("ctrl").to(device)
            tokenizer = CTRLTokenizer.from_pretrained("ctrl")
            saved_path = "completed_dataset_CTRL.csv"
        case "Reformer":
            model = ReformerModelWithLMHead.from_pretrained("google/reformer-crime-and-punishment").to(device)
            tokenizer = ReformerTokenizer.from_pretrained("google/reformer-crime-and-punishment")
            saved_path = "completed_dataset_Reformer.csv"
        case "GPT_NEO":
            model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M").to(device)
            tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
            saved_path = "completed_dataset_GPT_NEO.csv"
        case "Alpaca":
            model = AutoModelForCausalLM.from_pretrained("chavinlo/alpaca-native").to(device)
            tokenizer = AutoTokenizer.from_pretrained("chavinlo/alpaca-native")
            saved_path = "completed_dataset_Alpaca.csv"
        case "GPT4ALL":
            model = AutoModelForCausalLM.from_pretrained("nomic-ai/gpt4all-j").to(device)
            tokenizer = AutoTokenizer.from_pretrained("nomic-ai/gpt4all-j")
            saved_path = "completed_dataset_GPT4ALL.csv"
        case _:
            raise ValueError("Invalid model name. Please choose from: GPT2, XLNet, BART, T5, Pegasus, CTRL, Reformer, GPT_NEO, Alpaca, GPT4ALL")
            return None, None, None
    return model, tokenizer, saved_path
# Arguments: model_name, running device, dataset
def Complete_by_model(model_name, device, dataset):
    # Load pre-trained model and tokenizer
    model, tokenizer, saved_path = get_model_token(model_name, device)
    if model is None:
        return
    
    # Function to generate sentence completions without duplicating x_i and stopping at the first period
    def complete_sentence(input_text):
        inputs = tokenizer(input_text, return_tensors="pt").to(device)
        outputs = model.generate(inputs["input_ids"], max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
        completed_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # # Monitor the progress with percentage
        # print(f"Input: {input_text}")
        # print(f"Output: {completed_sentence}\n")

        # Remove the original input_text part from the generated completion
        completion_only = completed_sentence[len(input_text):].strip()
        
        # Remove all \n characters
        completion_only = completion_only.replace('\n', ' ')
        
        # Find the first period, and if not found, find the first comma
        first_period_index = completion_only.find('.')
        if first_period_index != -1 and first_period_index != 0:
            completion_only = completion_only[:first_period_index + 1]  # Include the period
        else:
            # Find the position of the second comma
            commas = [pos for pos, char in enumerate(completion_only) if char == ',']
            if len(commas) >= 2:
                completion_only = completion_only[:commas[1] + 1]  # Sentence more than two commas, then get the first two
            elif len(commas) == 1:
                    completion_only = completion_only[:commas[0] + 1]  # Sentence with only one comma, then get the first one
        return completion_only
    # Apply the completion process to each sentence in the x_i column
    tqdm.pandas()  # Add progress bar
    dataset['x_j'] = dataset['x_i'].progress_apply(complete_sentence)

    # Save the completed dataset
    dataset.to_csv(saved_path, index=False)


#### Complete the dataset using different models
# Clear GPU cache
torch.cuda.empty_cache()

# Complete_by_model("GPT2", device, cleaned_df) # Accepable completions
# Complete_by_model("XLNet", device, cleaned_df)    # Nonsensical completions
# Complete_by_model("BART", device, cleaned_df)    # Nonsensical completions
# Complete_by_model("T5", device, cleaned_df)       # Nonsensical completions
# Complete_by_model("Pegasus", device, cleaned_df)  # Backup option
# Complete_by_model("CTRL", device, cleaned_df)   # Nonsensical completions
Complete_by_model("Reformer", device, cleaned_df) # Accepable completions
# Complete_by_model("GPT_NEO", device, cleaned_df)  # Acceptable completions
# Complete_by_model("Alpaca", device, cleaned_df)   # CUDA out of memory (24GB needed)
#Complete_by_model("GPT4ALL", device, cleaned_df)   # CUDA out of memory (24GB needed)




c:\Users\YEH\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
  3%|▎         | 168/5000 [00:57<29:44,  2.71it/s]